In [32]:
import pandas as pd
data = pd.read_parquet('/drive2/phatnt/zTrans/data/data_method_tresitter1.parquet')
# data = data[data['id'] == 1245]
print(data.iloc[2])

id                                                                      7
fromLib                                                           ant:ant
toLib                                                  org.apache.ant:ant
repoName                                              bobmcwhirter_drools
prevCommit                       2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f
startCommit                      92384035a8651b675c82689b24837eb8adb81d66
endCommit                        0df25aead759b3b542a685ac21a3a009bcd22fe9
fileName                drools-compiler/src/test/java/org/drools/integ...
startCode               package org.drools.integrationtests;\n\nimport...
endCode                 package org.drools.integrationtests;\n\nimport...
startCommitChanges                               +org.mockito:mockito-all
endCommitChanges                -org.jmock:jmock\n-org.jmock:jmock-legacy
startCode_cleaned       package org.drools.integrationtests;\nimport j...
endCode_cleaned         package org.dr

In [ ]:
def remove_comments(source):
    source = source.replace('\r\n', '\n')
    state = "ETC"
    i = 0
    comments = []
    current_comment = None

    while i + 1 < len(source):
        if state == "ETC" and source[i] == '/' and source[i + 1] == '/':
            state = "LINE_COMMENT"
            current_comment = {
                "type": "LineComment",
                "range": {"start": i}
            }
            i += 2
            continue

        if state == "LINE_COMMENT" and source[i] == '\n':
            state = "ETC"
            current_comment["range"]["end"] = i
            comments.append(current_comment)
            current_comment = None
            i += 1
            continue

        if state == "ETC" and source[i] == '/' and source[i + 1] == '*':
            state = "BLOCK_COMMENT"
            current_comment = {
                "type": "BlockComment",
                "range": {"start": i}
            }
            i += 2
            continue

        if state == "BLOCK_COMMENT" and source[i] == '*' and source[i + 1] == '/':
            state = "ETC"
            current_comment["range"]["end"] = i + 2
            comments.append(current_comment)
            current_comment = None
            i += 2
            continue

        i += 1

    # Handle unfinished line comment at the end of the source code
    if current_comment and current_comment["type"] == "LineComment":
        if source[-1] == '\n':
            current_comment["range"]["end"] = len(source) - 1
        else:
            current_comment["range"]["end"] = len(source)
        comments.append(current_comment)

    # Remove the comments from the source code
    def remove_content(source, comments):
        result = []
        last_index = 0

        for comment in comments:
            start = comment["range"]["start"]
            end = comment["range"]["end"]
            # Append code before the comment
            result.append(source[last_index:start])
            # Update last index to skip the comment
            last_index = end

        # Append the remaining source code after the last comment
        result.append(source[last_index:])
        # Join and split lines, then filter out blank lines
        cleaned_code = ''.join(result).split('\n')
        return '\n'.join(line for line in cleaned_code if line.strip())

    return remove_content(source, comments)

In [ ]:

from tree_sitter import Language, Parser
import tree_sitter_java as tsjava

# Initialize the parser and set the Java language
JAVA_LANGUAGE = Language(tsjava.language())
parser = Parser(JAVA_LANGUAGE)
def extract_methods_with_body(java_code):
    methods = []
    # print('java_code :',java_code)
    try:
        try:
            java_code = java_code.encode('utf-8')
        except Exception as e:
            print(e)
        tree = parser.parse(java_code)
        root_node = tree.root_node
        # Function to extract code from a node
        def extract_code(source_code, node):
            start_byte = node.start_byte
            end_byte = node.end_byte
            return source_code[start_byte:end_byte].decode("utf-8")

        # Traverse the syntax tree and find the method declaration
        for child in root_node.children:
            if child.type == "class_declaration":
                for class_child in child.children:
                    if class_child.type == "class_body":
                        for body_child in class_child.children:
                            if body_child.type == "method_declaration":
                                # Check for valid method declaration without errors
                                if not any(c.type == "ERROR" for c in body_child.children):
                                    method_name = ""
                                    method_signature = ""
                                    method_body = ""
                                    modifiers = []
                                    return_type = ""

                                    # Extract components of the method declaration
                                    for method_child in body_child.children:
                                        if method_child.type == "modifiers":
                                            modifiers = [extract_code(java_code, modifier) for modifier in method_child.children]
                                        elif method_child.type in ["type", "type_identifier", "scoped_type_identifier"]:  # Handle nested type nodes
                                            return_type = extract_code(java_code, method_child)
                                        elif method_child.type == "identifier":  # Capture method name
                                            method_name = extract_code(java_code, method_child)
                                        elif method_child.type == "formal_parameters":  # Capture parameters
                                            parameters = extract_code(java_code, method_child)
                                            method_signature = f"{' '.join(modifiers)} {return_type} {method_name}{parameters}"
                                    method_body = extract_code(java_code, body_child)

                                    # Add the method details to the list
                                    methods.append({
                                        "name": method_name,
                                        "signature": method_signature.strip(),
                                        "body": method_body
                                    })
                                    
                                    
                                    # print(method_declaration_code)
                                    # methods.append(method_declaration_code)
            else:
                print(child.type)
        return methods
    except Exception as e:
        print('Loi :' ,e)
        return None
java_code = """
package org.drools.common;

/*
 * Copyright 2005 JBoss Inc
 * 
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 * 
 *      http://www.apache.org/licenses/LICENSE-2.0
 * 
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

import org.drools.FactHandle;
import org.drools.reteoo.LeftTuple;
import org.drools.reteoo.RightTuple;
import org.drools.runtime.rule.WorkingMemoryEntryPoint;

public interface InternalFactHandle
    extends
    FactHandle, Cloneable {
    public int getId();

    public long getRecency();

    public Object getObject();

    public void setObject(Object object);

    public void setEqualityKey(EqualityKey key);

    public EqualityKey getEqualityKey();

    public void setRecency(long recency);

    public void invalidate();
    
    public boolean isValid();
    
    public int getIdentityHashCode();

    public int getObjectHashCode();
    
    /**
     * Returns true if this FactHandle represents
     * and Event or false if this FactHandle represents
     * a regular Fact
     * 
     * @return
     */
    public boolean isEvent();
    
    public RightTuple getFirstRightTuple(){
        print(a)
        };

    public void setFirstRightTuple(RightTuple rightTuple);

    public RightTuple getLastRightTuple();

    public void setLastRightTuple(RightTuple rightTuple);    
        
    public void setFirstLeftTuple(LeftTuple leftTuple);
    
    public LeftTuple getFirstLeftTuple();
    
    public void setLastLeftTuple(LeftTuple leftTuple);    
    
    public LeftTuple getLastLeftTuple();
    
    public WorkingMemoryEntryPoint getEntryPoint();
    
    public void setEntryPoint( WorkingMemoryEntryPoint ep );
    
    public InternalFactHandle clone();
    
    public String toExternalForm();
    
    protected void tearDown() throws Exception {
        super.tearDown();
    }

    
}
"""
methods = extract_methods_with_body(java_code)
print('methods :',methods)
# Hiển thị kết quả
for method in methods:
    print(method)
    print("-" * 40)

package_declaration
block_comment
import_declaration
import_declaration
import_declaration
import_declaration
interface_declaration
methods : []


: 

In [30]:
def diff_methods(methods_start, methods_end):
    """
    Compare methods based on their full dictionaries (e.g., name, signature, body).
    """
    # Normalize methods for comparison
    normalized_start = [{key: method[key].strip() if isinstance(method[key], str) else method[key] for key in method} for method in methods_start]
    normalized_end = [{key: method[key].strip() if isinstance(method[key], str) else method[key] for key in method} for method in methods_end]

    # Convert lists of methods to sets of frozensets for comparison
    set_start = set(frozenset(item.items()) for item in normalized_start)
    set_end = set(frozenset(item.items()) for item in normalized_end)

    # Determine differences
    removed_methods = [dict(items) for items in (set_start - set_end)]  # Methods in start but not in end
    added_methods = [dict(items) for items in (set_end - set_start)]    # Methods in end but not in start
    unchanged_methods = [dict(items) for items in (set_start & set_end)]  # Methods in both

    return {
        "removed": removed_methods,
        "added": added_methods,
        "unchanged": unchanged_methods,
    }


def check_same_methods(method1, method2):
    name_method1 = method1.split('(')[0]
    name_method2 = method2.split('(')[0]
    return name_method1 == name_method2
# print('Start Code :',data.iloc[1]['startCode'])
methods_start = extract_methods_with_body(data.iloc[3]['startCode'])
# for i in methods_start:
#     print('name :',i['name'])
#     print('signature :',i['signature'])
#     print('body :',i['body'])
#     print('----->')
methods_end = extract_methods_with_body(data.iloc[3]['endCode'])
diff_result = diff_methods(methods_start, methods_end)
row_changes = []
removed = diff_result["removed"]
added = diff_result["added"]

remaining_added = list(added)

for method1 in removed:
    for method2 in list(remaining_added):  # Duyệt qua một bản sao của remaining_added
        if check_same_methods(method1['body'], method2['body']):
            row_changes.append([method1, method2])
            remaining_added.remove(method2)  # Xóa method2 khỏi remaining_added
for containmethods in remaining_added:
    row_changes.append(['',containmethods])
for i in row_changes:
    for ex in i:
        print(ex)
        print('****')
    print('----->')

# Hiển thị kết quả
print("Removed Methods:")
for method in diff_result["removed"]:
    print(method)
print("-" * 40)

print("Added Methods:")
for method in diff_result["added"]:
    print(method)
print("-" * 40)


{'name': 'testWorkingMemoryLoggerWithUnbalancedBranches', 'signature': 'public  testWorkingMemoryLoggerWithUnbalancedBranches()', 'body': 'public void testWorkingMemoryLoggerWithUnbalancedBranches() throws Exception {\n        final PackageBuilder builder = new PackageBuilder();\n        builder.addPackageFromDrl( new InputStreamReader( getClass().getResourceAsStream( "test_Logger.drl" ) ) );\n        final Package pkg = builder.getPackage();\n\n        RuleBase ruleBase = getRuleBase();\n        ruleBase.addPackage( pkg );\n        ruleBase = SerializationHelper.serializeObject( ruleBase );\n\n        final WorkingMemory wm = ruleBase.newStatefulSession();\n\n        try {\n            final WorkingMemoryFileLogger logger = new WorkingMemoryFileLogger( wm );\n            logger.setFileName( "testLogger" );\n\n            wm.fireAllRules();\n\n            wm.insert( new Cheese( "a",\n                                   10 ) );\n            wm.insert( new Cheese( "b",\n                  

In [4]:
import pandas as pd

def process_and_expand_data(data):
    """
    Hàm xử lý và mở rộng DataFrame gốc bằng cách thêm các cột methods_before và methods_after.
    """
    expanded_data = []
    count = 0

    for index, row in data.iterrows():
        try:

            startCode = row['startCode_cleaned']
            endCode = row['endCode_cleaned']


            methods_start = extract_methods_with_body(startCode)
            methods_end = extract_methods_with_body(endCode)


            diff_result = diff_methods(methods_start, methods_end)
            removed = diff_result["removed"]
            added = diff_result["added"]

            row_changes = []
            remaining_added = list(added)

            for method1 in removed:
                have_func = False
                for method2 in list(remaining_added):
                    if check_same_methods(method1, method2):
                        row_changes.append([method1, method2])
                        remaining_added.remove(method2)
                        have_func = True
                        break
                if have_func == False:
                    row_changes.append([method1,''])

            for containmethods in remaining_added:
                row_changes.append(['',containmethods])


            for method1, method2 in row_changes:
                expanded_data.append({
                    **row.to_dict(),
                    'methods_before': method1,
                    'methods_after': method2
                })

        # for method1 in removed:
        #     if not any(method1 == change[0] for change in row_changes):
        #         expanded_data.append({
        #             **row.to_dict(),
        #             'methods_before': method1,
        #             'methods_after': ''
        #         })

        # for method2 in remaining_added:
        #     expanded_data.append({
        #         **row.to_dict(),
        #         'methods_before': '',
        #         'methods_after': method2
        #     })
        except Exception as e:
            print(e)
            # import traceback
            # traceback.print_exc()
            print('Err at index:',count)
            print('--->')
        count+=1

    expanded_df = pd.DataFrame(expanded_data)
    return expanded_df


expanded_data = process_and_expand_data(data)

print(expanded_data)

Could not process token at "\", line 120: " insert( $abc );\n"
Could not process token at "\", line 124: " insert( $abc );\n"
'NoneType' object is not iterable
Err at index: 8
--->
Unterminated character/string literal at """, line 362: this.pkg = new Package( "org.drools.examples.manners" );
'NoneType' object is not iterable
Err at index: 88
--->
Could not process token at "\", line 51: str += "   line2\n";
'NoneType' object is not iterable
Err at index: 89
--->


KeyboardInterrupt: 

In [10]:
expanded_data

,id,fromLib,toLib,repoName,prevCommit,startCommit,endCommit,fileName,startCommitChanges,endCommitChanges,startCode_cleaned,endCode_cleaned,diff_cleaned,methods_before,methods_after,methods_diff
0,1245,com.google.code.gson:gson,com.fasterxml.jackson.core:jackson-core,glowroot_glowroot,1fefb80c27fb37bb07553a3ac8299313cc2235d0,a385001cd79dba7be366ce0a854cb1320ea088f3,a385001cd79dba7be366ce0a854cb1320ea088f3,testkit/src/main/java/io/informant/testkit/Gen...,+com.fasterxml.jackson.core:jackson-core\n+com...,+com.fasterxml.jackson.core:jackson-core\n+com...,package io.informant.testkit;\nimport checkers...,package io.informant.testkit;\nimport checkers...,"--- \n+++ \n@@ -10,7 +10,7 @@\n private in...",public String toString() {\n return Obj...,public String toString() {\n return Obj...,
1,1245,com.google.code.gson:gson,com.fasterxml.jackson.core:jackson-core,glowroot_glowroot,1fefb80c27fb37bb07553a3ac8299313cc2235d0,a385001cd79dba7be366ce0a854cb1320ea088f3,a385001cd79dba7be366ce0a854cb1320ea088f3,testkit/src/main/java/io/informant/testkit/Gen...,+com.fasterxml.jackson.core:jackson-core\n+com...,+com.fasterxml.jackson.core:jackson-core\n+com...,package io.informant.testkit;\nimport checkers...,package io.informant.testkit;\nimport checkers...,"--- \n+++ \n@@ -10,7 +10,7 @@\n private in...",public void setVersionHash(String versionHash)...,,
2,1245,com.google.code.gson:gson,com.fasterxml.jackson.core:jackson-core,glowroot_glowroot,1fefb80c27fb37bb07553a3ac8299313cc2235d0,a385001cd79dba7be366ce0a854cb1320ea088f3,a385001cd79dba7be366ce0a854cb1320ea088f3,testkit/src/main/java/io/informant/testkit/Gen...,+com.fasterxml.jackson.core:jackson-core\n+com...,+com.fasterxml.jackson.core:jackson-core\n+com...,package io.informant.testkit;\nimport checkers...,package io.informant.testkit;\nimport checkers...,"--- \n+++ \n@@ -10,7 +10,7 @@\n private in...",public String getVersionHash() {\n retu...,,
3,1245,com.google.code.gson:gson,com.fasterxml.jackson.core:jackson-core,glowroot_glowroot,1fefb80c27fb37bb07553a3ac8299313cc2235d0,a385001cd79dba7be366ce0a854cb1320ea088f3,a385001cd79dba7be366ce0a854cb1320ea088f3,testkit/src/main/java/io/informant/testkit/Gen...,+com.fasterxml.jackson.core:jackson-core\n+com...,+com.fasterxml.jackson.core:jackson-core\n+com...,package io.informant.testkit;\nimport checkers...,package io.informant.testkit;\nimport checkers...,"--- \n+++ \n@@ -10,7 +10,7 @@\n private in...",,public String getVersion() {\n return v...,
4,1245,com.google.code.gson:gson,com.fasterxml.jackson.core:jackson-core,glowroot_glowroot,1fefb80c27fb37bb07553a3ac8299313cc2235d0,a385001cd79dba7be366ce0a854cb1320ea088f3,a385001cd79dba7be366ce0a854cb1320ea088f3,testkit/src/main/java/io/informant/testkit/Gen...,+com.fasterxml.jackson.core:jackson-core\n+com...,+com.fasterxml.jackson.core:jackson-core\n+com...,package io.informant.testkit;\nimport checkers...,package io.informant.testkit;\nimport checkers...,"--- \n+++ \n@@ -10,7 +10,7 @@\n private in...",,void setVersion(@Nullable String version) {\n ...,


In [ ]:
expanded_data['methods_before'] = expanded_data['methods_before'].apply(lambda x: remove_comments(x))
expanded_data['methods_after'] = expanded_data['methods_after'].apply(lambda x: remove_comments(x))

In [8]:
expanded_data.to_parquet('/drive2/phatnt/zTrans/data/data_method.parquet')